In [ ]:
import openvino as ov
import pandas as pd
import os
import numpy as np
import gc
import pickle
import warnings
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import TensorDataset, DataLoader
import nncf

warnings.filterwarnings('ignore')

Import dataset pickle file

In [ ]:
os.chdir('../..')

def get_pickle(pickle_dir, file_name):
    pickle_dir = os.path.join(os.getcwd(), 'datasets/fsc22/Pickle Files/' + pickle_dir)
    fold_dir = os.path.join(pickle_dir, file_name)
    infile = open(fold_dir, 'rb')
    fold = pickle.load(infile)
    infile.close()

    return fold

In [ ]:
def get_train_vaild_data():
    # enter pickle file directory name, pickle filename for the fold
    valid_spects = get_pickle('aug_ts_ps_mixed_features_5_20', 'aug_ts_ps_mixed_features_5_20_fold1') # Change pickle names accordingly

    valid_df = pd.DataFrame(valid_spects, columns=['feature', 'class'])

    del valid_spects

    gc.collect()

    X_valid_cv = np.array(valid_df['feature'].tolist())
    y_valid_cv = np.array(valid_df['class'].tolist())

    return X_valid_cv, y_valid_cv

X_valid_cv, y_valid_cv = get_train_vaild_data()

print(f'X_valid_cv shape: {np.shape(X_valid_cv)}')
print(f'y_valid_cv shape: {np.shape(y_valid_cv)}')

Create torch dataloader

In [ ]:
def transform_fn(data_item):
    images, _ = data_item
    return images

# Convert NumPy arrays to PyTorch tensors
X_tensor = torch.Tensor(X_valid_cv)
y_tensor = torch.Tensor(y_valid_cv)

# Create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Specify batch size for DataLoader
batch_size = 1

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

calibration_dataset = nncf.Dataset(dataloader, transform_fn)
validation_dataset = nncf.Dataset(dataloader, transform_fn)

Method for validation of quantized model

In [ ]:
def validate(model: ov.runtime.CompiledModel,
             validation_loader: torch.utils.data.DataLoader) -> float:
    predictions = []
    references = []

    output = model.outputs[0]

    for images, target in validation_loader:
        pred = model(images)[output]
        predictions.append(np.argmax(pred, axis=1))
        references.append(target)

    predictions = np.concatenate(predictions, axis=0)
    references = np.concatenate(references, axis=0)

    return accuracy_score(predictions, references)

Qunatize the model

In [ ]:
core = ov.Core()

# replace with the relative path of the xml file of the model to be pruned
ir_model_name_xml = 'models/MobileNetV3-Small/MobileNet_aug5_mel_5fold/MobileNet_aug5_mel_5fold_weight_pruned_0_filter_pruned_0_8_fold1.xml'

model = core.read_model(model=ir_model_name_xml)

quantized_model = nncf.quantize_with_accuracy_control(
    model,
    calibration_dataset=calibration_dataset,
    validation_dataset=validation_dataset,
    validation_fn=validate,
    max_drop=0.01
)

Save the quantized model as OpenVINO IR model

Change working directory to model directory

Save model without filter pruning mask

Enter the save filename of quantized model as output_model argument

In [ ]:
os.chdir('MobileNetV3-Small/MobileNet_aug5_mel_5fold')

ov.save_model(quantized_model, output_model="model/MobileNetV3-Small_aug5_mel_5fold_weight_pruned_0_9_filter_pruned_0_7_quantized_fold1.xml")

os.chdir('../..')